In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/My Drive/semeval_2022/dontpatronizeme_v1.3

/content/drive/My Drive/semeval_2022/dontpatronizeme_v1.3


In [ ]:
import random
import os
from urllib import request

In [ ]:

module_url = f"https://raw.githubusercontent.com/Perez-AlmendrosC/dontpatronizeme/master/semeval-2022/dont_patronize_me.py"
module_name = module_url.split('/')[-1]
print(f'Fetching {module_url}')
#with open("file_1.txt") as f1, open("file_2.txt") as f2
with request.urlopen(module_url) as f, open(module_name,'w') as outf:
  a = f.read()
  outf.write(a.decode('utf-8'))

Fetching https://raw.githubusercontent.com/Perez-AlmendrosC/dontpatronizeme/master/semeval-2022/dont_patronize_me.py


In [ ]:
from dont_patronize_me import DontPatronizeMe
# Initialize a dpm (Don't Patronize Me) object.
# It takes two arguments as input: 
# (1) Path to the training set, which is the root directory of this notebook.
# (2) Path to the test set, which will be released when the evaluation phase begins. 
# For now, you can just use the dataset for Subtask 1, which the code will load without labels.
dpm = DontPatronizeMe('.', 'dontpatronizeme_pcl.tsv')

In [ ]:
# This method loads the subtask 1 data
dpm.load_task1()
# which we can then access as a dataframe
dpm.train_task1_df.head(10)

,par_id,art_id,keyword,country,text,label,orig_label
0,1,@@24942188,hopeless,ph,"we 're living in times of absolute insanity , ...",0,0
1,2,@@21968160,migrant,gh,"in libya today , there are countless number of...",0,0
2,3,@@16584954,immigrant,ie,"""white house press secretary sean spicer said ...",0,0
3,4,@@7811231,disabled,nz,council customers only signs would be displaye...,0,0
4,5,@@1494111,refugee,ca,""""""" just like we received migrants fleeing el ...",0,0
5,6,@@9382277,in-need,in,"to bring down high blood sugar levels , insuli...",0,0
6,7,@@7562079,refugee,za,the european union is making an historic mista...,0,0
7,8,@@23663488,hopeless,nz,""""""" they 're either hopeless for being beaten ...",0,0
8,9,@@3449225,homeless,ph,"nueva era , ilocos norte - no family shall be ...",0,1
9,10,@@2485090,in-need,nz,his spokesman said the kremlin needed more inf...,0,0


In [ ]:
len(dpm.train_task1_df.index)

10469

In [ ]:

dpm.train_task1_df.loc[dpm.train_task1_df.label==0].count()

par_id        9476
art_id        9476
keyword       9476
country       9476
text          9476
label         9476
orig_label    9476
dtype: int64

In [ ]:
dpm.train_task1_df['text'][0]

'0'

In [ ]:
task1_test=dpm.train_task1_df.sample(frac=0.2)

In [ ]:
task1_train=dpm.train_task1_df.drop(task1_test.index)

In [ ]:
task1_train.loc[task1_train.label==1].count()

par_id        785
art_id        785
keyword       785
country       785
text          785
label         785
orig_label    785
dtype: int64

In [ ]:
task1_test.loc[task1_test.label==0].count()

par_id        1886
art_id        1886
keyword       1886
country       1886
text          1886
label         1886
orig_label    1886
dtype: int64

In [ ]:
len(task1_test)

2094

In [ ]:
task1_train.to_csv('task1_train.csv')

In [ ]:
task1_test.to_csv('task1_test.csv')

In [ ]:
from pathlib import Path
def read_imdb_split(split_dir):
    split_dir = Path(split_dir)
    texts = []
    labels = []
    for label_dir in ["pos", "neg"]:
        for text_file in (split_dir/label_dir).iterdir():
            texts.append(text_file.read_text())
            labels.append(0 if label_dir is "neg" else 1)

    return texts, labels

train_texts, train_labels = read_imdb_split('aclImdb/train')
test_texts, test_labels = read_imdb_split('aclImdb/test')

In [ ]:
from sklearn.model_selection import train_test_split
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=.2)

In [ ]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [ ]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

In [ ]:
import torch

class IMDbDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = IMDbDataset(train_encodings, train_labels)
val_dataset = IMDbDataset(val_encodings, val_labels)
test_dataset = IMDbDataset(test_encodings, test_labels)

In [ ]:

from torch.utils.data import DataLoader
from transformers import DistilBertForSequenceClassification, AdamW

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')
model.to(device)
model.train()

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

optim = AdamW(model.parameters(), lr=5e-5)

for epoch in range(3):
    for batch in train_loader:
        optim.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs[0]
        loss.backward()
        optim.step()


In [ ]:
class RobertaClass(torch.nn.Module):
    def __init__(self):
        super(RobertaClass, self).__init__()
        self.l1 = RobertaModel.from_pretrained("roberta-base")
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.3)
        self.classifier = torch.nn.Linear(768, 2)

    def forward(self, input_ids, attention_mask, token_type_ids):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output

model = RobertaClass()
model.to(device)

In [ ]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)
optimizer = torch.optim.Adam(params =  model.parameters(), lr=lr)

In [ ]:
def calcuate_accuracy(preds, targets):
    n_correct = (preds==targets).sum().item()
    return n_correct

In [ ]:
def train(epoch):
    tr_loss = 0
    n_correct = 0
    nb_tr_steps = 0
    nb_tr_examples = 0
    model.train()
    for _,data in tqdm(enumerate(training_loader, 0)):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.long)

        outputs = model(ids, mask, token_type_ids)
        loss = loss_function(outputs, targets)
        tr_loss += loss.item()
        big_val, big_idx = torch.max(outputs.data, dim=1)
        n_correct += calcuate_accuracy(big_idx, targets)

        nb_tr_steps += 1
        nb_tr_examples+=targets.size(0)
        
        if _%5000==0:
            loss_step = tr_loss/nb_tr_steps
            accu_step = (n_correct*100)/nb_tr_examples 
            print(f"Training Loss per 5000 steps: {loss_step}")
            print(f"Training Accuracy per 5000 steps: {accu_step}")

        optimizer.zero_grad()
        loss.backward()
        # # When using GPU
        optimizer.step()

    print(f'The Total Accuracy for Epoch {epoch}: {(n_correct*100)/nb_tr_examples}')
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Training Loss Epoch: {epoch_loss}")
    print(f"Training Accuracy Epoch: {epoch_accu}")

    return

In [ ]:
for epoch in range(epochs):
    train(epoch)

In [ ]:
def valid(model, testing_loader):
    model.eval()
    n_correct = 0; n_wrong = 0; total = 0; tr_loss=0; nb_tr_steps=0; nb_tr_examples=0
    with torch.no_grad():
        for _, data in tqdm(enumerate(testing_loader, 0)):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)
            targets = data['targets'].to(device, dtype = torch.long)
            outputs = model(ids, mask, token_type_ids).squeeze()
            loss = loss_function(outputs, targets)
            tr_loss += loss.item()
            big_val, big_idx = torch.max(outputs.data, dim=1)
            n_correct += calcuate_accuracy(big_idx, targets)

            nb_tr_steps += 1
            nb_tr_examples+=targets.size(0)
            
            if _%5000==0:
                loss_step = tr_loss/nb_tr_steps
                accu_step = (n_correct*100)/nb_tr_examples
                print(f"Validation Loss per 100 steps: {loss_step}")
                print(f"Validation Accuracy per 100 steps: {accu_step}")
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Validation Loss Epoch: {epoch_loss}")
    print(f"Validation Accuracy Epoch: {epoch_accu}")
    
    return epoch_accu

In [ ]:
acc = valid(model, testing_loader)
print("Accuracy on test data = %0.2f%%" % acc)

In [ ]:

output_model_file = 'pytorch_roberta_sentiment.bin'
output_vocab_file = './'

model_to_save = model
torch.save(model_to_save, output_model_file)
tokenizer.save_vocabulary(output_vocab_file)

print('All files saved')
print('This tutorial is completed')

In [ ]:
###################################   TASK 2 ###########

In [ ]:
import pandas as pd

In [ ]:

# Same for subtask 2
dpm.load_task2(return_one_hot=True)
# which we can then access as a dataframe
dpm.train_task2_df.head(10)

,par_id,art_id,text,keyword,country,label
0,4046,@@14767805,we also know that they can benefit by receivin...,hopeless,us,"[1, 1, 0, 0, 0, 0, 0]"
1,1279,@@7896098,pope francis washed and kissed the feet of mus...,refugee,ng,"[0, 0, 1, 0, 0, 0, 0]"
2,8330,@@17252299,many refugees do n't want to be resettled anyw...,refugee,ng,"[0, 0, 0, 1, 0, 0, 0]"
3,4063,@@3002894,"""budding chefs , like """" fred """" , """" winston ...",in-need,ie,"[1, 1, 0, 0, 1, 1, 0]"
4,4089,@@25597822,"""in a 90-degree view of his constituency , one...",homeless,pk,"[0, 1, 0, 0, 0, 0, 0]"
5,432,@@15802146,he depicts demonstrations by refugees at the b...,refugee,nz,"[0, 0, 0, 0, 1, 0, 0]"
6,4177,@@930041,the word of god is truth that 's living and ab...,hopeless,us,"[0, 1, 0, 0, 1, 0, 0]"
7,3963,@@18867357,"chantelle owens , mrs planet 2016 , hosted the...",in-need,za,"[0, 1, 1, 0, 1, 0, 0]"
8,2001,@@14012804,t is remiss not to mention here that not all s...,poor-families,tz,"[0, 0, 0, 1, 0, 0, 0]"
9,369,@@15636898,""""""" people do n't understand the hurt , people...",women,ie,"[1, 1, 0, 1, 1, 0, 0]"


In [ ]:
dpm.train_task2_df['label'][0]

array([1, 1, 0, 0, 0, 0, 0])

In [ ]:
len(dpm.train_task2_df.index)

993

In [ ]:
task2_train.loc[task1_train.labels==1].count()

NameError: ignored

In [ ]:
task2_test=dpm.train_task2_df.sample(frac=0.2)

In [ ]:
task2_test=task2_test.reset_index(drop=True)

In [ ]:
task2_test

,par_id,art_id,text,keyword,country,label
0,10400,@@25998036,""""""" they 're a little bit behind me i think , ...",homeless,pk,"[0, 1, 1, 0, 0, 0, 0]"
1,2658,@@25796035,"by this exercise , the prime minister would be...",poor-families,lk,"[1, 1, 0, 1, 0, 0, 0]"
2,1908,@@2597332,inspiration that anything is possible . you ca...,immigrant,ph,"[1, 1, 0, 1, 1, 0, 0]"
3,8811,@@24844729,""""""" they want to live in a country that treats...",women,ie,"[1, 1, 0, 0, 0, 0, 0]"
4,9961,@@1952746,""""""" these children are vulnerable and need to ...",vulnerable,ke,"[1, 1, 0, 0, 0, 1, 0]"
...,...,...,...,...,...,...
194,2405,@@1415929,we think it 's lovely that so many have come f...,in-need,us,"[0, 1, 0, 0, 0, 0, 0]"
195,6420,@@20370953,"""homeless children are """" living like refugees...",refugee,ie,"[0, 0, 0, 1, 1, 1, 0]"
196,7790,@@1872744,"""as a child , i have always been told to give ...",in-need,gh,"[0, 1, 0, 0, 1, 0, 0]"
197,7726,@@4749950,"""in dublin , the church of ireland archbishop ...",refugee,ie,"[0, 1, 0, 0, 1, 0, 0]"


In [ ]:
task2_train=dpm.train_task2_df.drop(task2_test.index)

In [ ]:
task2_train=task2_train.reset_index(drop=True)

In [ ]:
task2_test=pd.read_csv('task2_test.csv')

In [ ]:
task2_test

,Unnamed: 0,par_id,art_id,text,keyword,country,label,upr,ss,ps,av,m,c,pm
0,0,10400,@@25998036,""""""" they 're a little bit behind me i think , ...",homeless,pk,[0 1 1 0 0 0 0],0,1,1,0,0,0,0
1,1,2658,@@25796035,"by this exercise , the prime minister would be...",poor-families,lk,[1 1 0 1 0 0 0],1,1,0,1,0,0,0
2,2,1908,@@2597332,inspiration that anything is possible . you ca...,immigrant,ph,[1 1 0 1 1 0 0],1,1,0,1,1,0,0
3,3,8811,@@24844729,""""""" they want to live in a country that treats...",women,ie,[1 1 0 0 0 0 0],1,1,0,0,0,0,0
4,4,9961,@@1952746,""""""" these children are vulnerable and need to ...",vulnerable,ke,[1 1 0 0 0 1 0],1,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194,194,2405,@@1415929,we think it 's lovely that so many have come f...,in-need,us,[0 1 0 0 0 0 0],0,1,0,0,0,0,0
195,195,6420,@@20370953,"""homeless children are """" living like refugees...",refugee,ie,[0 0 0 1 1 1 0],0,0,0,1,1,1,0
196,196,7790,@@1872744,"""as a child , i have always been told to give ...",in-need,gh,[0 1 0 0 1 0 0],0,1,0,0,1,0,0
197,197,7726,@@4749950,"""in dublin , the church of ireland archbishop ...",refugee,ie,[0 1 0 0 1 0 0],0,1,0,0,1,0,0


In [ ]:
target_cols=['upr','ss','ps','av','m','c','pm']

In [ ]:
len(task2_test)

199

In [ ]:
upr=0
ss=0
ps=0
av=0
m=0
c=0
pm=0
for i in task2_test[target_cols].values:
  if i[0]==1:
    upr+=1
  if i[1]==1:
    ss+=1
  if i[2]==1:
    ps+=1
  if i[3]==1:
    av+=1
  if i[4]==1:
    m+=1
  if i[5]==1:
    c+=1
  if i[6]==1:
    pm+=1
print('unbalanced power relations: %4f, shallow solution: %4f, presupposition: %4f, authority voice: %4f, metaphor: %4f, compassion: %4f, the poorer the merrier: %4f' % (upr,ss,ps,av,m,c,pm))

unbalanced power relations: 44.000000, shallow solution: 141.000000, presupposition: 31.000000, authority voice: 45.000000, metaphor: 101.000000, compassion: 39.000000, the poorer the merrier: 6.000000


In [ ]:
s

570

In [ ]:
len(task2_test)

199

In [ ]:
label_list=['upr','ss','ps','av','m','c','pm']

In [ ]:
for i in label_list:
  task2_test[i]=0

In [ ]:
task2_test

,par_id,art_id,text,keyword,country,label,upr,ss,ps,av,m,c,pm
0,10400,@@25998036,""""""" they 're a little bit behind me i think , ...",homeless,pk,"[0, 1, 1, 0, 0, 0, 0]",0,0,0,0,0,0,0
1,2658,@@25796035,"by this exercise , the prime minister would be...",poor-families,lk,"[1, 1, 0, 1, 0, 0, 0]",0,0,0,0,0,0,0
2,1908,@@2597332,inspiration that anything is possible . you ca...,immigrant,ph,"[1, 1, 0, 1, 1, 0, 0]",0,0,0,0,0,0,0
3,8811,@@24844729,""""""" they want to live in a country that treats...",women,ie,"[1, 1, 0, 0, 0, 0, 0]",0,0,0,0,0,0,0
4,9961,@@1952746,""""""" these children are vulnerable and need to ...",vulnerable,ke,"[1, 1, 0, 0, 0, 1, 0]",0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
194,2405,@@1415929,we think it 's lovely that so many have come f...,in-need,us,"[0, 1, 0, 0, 0, 0, 0]",0,0,0,0,0,0,0
195,6420,@@20370953,"""homeless children are """" living like refugees...",refugee,ie,"[0, 0, 0, 1, 1, 1, 0]",0,0,0,0,0,0,0
196,7790,@@1872744,"""as a child , i have always been told to give ...",in-need,gh,"[0, 1, 0, 0, 1, 0, 0]",0,0,0,0,0,0,0
197,7726,@@4749950,"""in dublin , the church of ireland archbishop ...",refugee,ie,"[0, 1, 0, 0, 1, 0, 0]",0,0,0,0,0,0,0


In [ ]:

cnt=0
for i in task2_test.label:
  for id,name in enumerate(label_list):
    if i[id]==1:
      task2_test.loc[cnt,name]=1
    else:
      task2_test.loc[cnt,name]=0
  cnt+=1


In [ ]:
task2_test

,par_id,art_id,text,keyword,country,label,upr,ss,ps,av,m,c,pm
0,10400,@@25998036,""""""" they 're a little bit behind me i think , ...",homeless,pk,"[0, 1, 1, 0, 0, 0, 0]",0,1,1,0,0,0,0
1,2658,@@25796035,"by this exercise , the prime minister would be...",poor-families,lk,"[1, 1, 0, 1, 0, 0, 0]",1,1,0,1,0,0,0
2,1908,@@2597332,inspiration that anything is possible . you ca...,immigrant,ph,"[1, 1, 0, 1, 1, 0, 0]",1,1,0,1,1,0,0
3,8811,@@24844729,""""""" they want to live in a country that treats...",women,ie,"[1, 1, 0, 0, 0, 0, 0]",1,1,0,0,0,0,0
4,9961,@@1952746,""""""" these children are vulnerable and need to ...",vulnerable,ke,"[1, 1, 0, 0, 0, 1, 0]",1,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
194,2405,@@1415929,we think it 's lovely that so many have come f...,in-need,us,"[0, 1, 0, 0, 0, 0, 0]",0,1,0,0,0,0,0
195,6420,@@20370953,"""homeless children are """" living like refugees...",refugee,ie,"[0, 0, 0, 1, 1, 1, 0]",0,0,0,1,1,1,0
196,7790,@@1872744,"""as a child , i have always been told to give ...",in-need,gh,"[0, 1, 0, 0, 1, 0, 0]",0,1,0,0,1,0,0
197,7726,@@4749950,"""in dublin , the church of ireland archbishop ...",refugee,ie,"[0, 1, 0, 0, 1, 0, 0]",0,1,0,0,1,0,0


In [ ]:
task2_test.to_csv('task2_test.csv')


In [ ]:
task2_train.to_csv('task2_train.csv')